In [1]:
# # adapting simon's older PC-based 1dvar code to read in GMI 
# data and run oceanic retrievals. started DD 17/05/18
#   - - working as wv retrieval 03/06/18
#  playing with surface retrieval (jacobians first), DD late august 2018

# Set up ARTS/Python environment


In [2]:
%env ARTS_INCLUDE_PATH=/home/dudavid/arts/controlfiles/
%env ARTS_DATA_PATH=/home/dudavid/arts/arts-xml/
%env ARTS_BUILD_PATH=/home/dudavid/arts/build/

%matplotlib inline
import glob
from h5py import File

from typhon.arts.workspace import Workspace, arts_agenda
ws = Workspace(verbosity=0)
ws.execute_controlfile("general/general.arts")
ws.execute_controlfile("general/continua.arts")
ws.execute_controlfile("general/agendas.arts")
ws.execute_controlfile("general/planet_earth.arts")

env: ARTS_INCLUDE_PATH=/home/dudavid/arts/controlfiles/
env: ARTS_DATA_PATH=/home/dudavid/arts/arts-xml/
env: ARTS_BUILD_PATH=/home/dudavid/arts/build/


/home/dudavid/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Loading ARTS API from: /home/dudavid/arts/build/src/libarts_api.so


In [3]:
from typhon.arts.workspace.variables import *

ws.Copy( ws.abs_xsec_agenda, ws.abs_xsec_agenda__noCIA )
ws.Copy( ws.iy_main_agenda, ws.iy_main_agenda__Emission )
ws.Copy( ws.iy_space_agenda, ws.iy_space_agenda__CosmicBackground )
ws.Copy( ws.propmat_clearsky_agenda, ws.propmat_clearsky_agenda__OnTheFly )
ws.Copy( ws.ppath_agenda, ws.ppath_agenda__FollowSensorLosPath )
ws.Copy( ws.ppath_step_agenda, ws.ppath_step_agenda__GeometricPath )

In [4]:
# define absorbing species and sensor (here using metmm library, used again below)
ws.abs_speciesSet(species=["H2O-PWR98","O2-PWR93","N2-SelfContStandardType"])
ws.abs_lines_per_speciesSetEmpty()

# General Settings

In [5]:
ws.stokes_dim = 2 # to get V and H pol out of metmm
ws.iy_unit = "PlanckBT" # equivalent: ws.StringSet( iy_unit, "PlanckBT" )

# Atmosphere set up # (take Psfc, T from model later)

In [6]:
ws.atmosphere_dim = 1  # for 1DVAR
p = np.array([1013.,975.,950.,925.,900.,850.,800.,750.,700.,650.,600.,550.,500.,400.,300.,200.,100.])*100.0
ws.p_grid = p[:] #0.5 * (p[1:] + p[:-1])
print(ws.p_grid.value)
ws.AtmRawRead( basename = "planets/Earth/Fascod/tropical/tropical") #tropical atmosphere assumed
ws.AtmosphereSet1D()
ws.AtmFieldsCalc()

ws.z_surface = np.asarray(ws.z_field)[0]
ws.t_surface = np.asarray(ws.t_field)[0]


[101300.  97500.  95000.  92500.  90000.  85000.  80000.  75000.  70000.
  65000.  60000.  55000.  50000.  40000.  30000.  20000.  10000.]


# Sensor definition -- GMI sensor setup w/ metmm

In [7]:
ws.ArrayOfIndexCreate("channels")
# initial sensor setup, all 13 channels at low frequency EIA
csub = [[x for x in range(9)], [x+9 for x in range(4)]] # 13 GMI channels total
#csub = [ [3,4,5,6], [11,12] ]#  define channel indices (0-8 10-89GHz, 9-12 166-183GHz)
nch = np.size(csub)
#ws.channels = csub #[0,:]+csub[1,:] #list(np.ravel(csub)) # 23.8 and 183 channels only for WV

ws.ArrayOfIndexCreate("viewing_angles") # necessary if just using one pencil beam angle?
#ws.viewing_angles = [1] # index of viewing angles in metmm file -- for GMI, 49.2, 52.8
# current setup is low freqs at index 1, high freqs at index 0

ws.sensor_pos  = np.array([[407e3]])#, -30.0, 180.0]]) # 407km alt, 30S, 180E
#can take exact alt from file if desired, same with lat/lon values (does that have effect?)
ws.sensor_time = np.array([0.0]) # same here
ws.sensor_los  = np.array([[180.0]]) # since angle taken care of in metmm file 
# zenith angle followed by azimuth. zenith=0 is straight up, 180 is straight down. azimuth=0 is N, positive to east 

ws.IndexCreate("met_mm_accuracy") 
ws.met_mm_accuracy = 1 # points within each bandwidth to comute abs characteristics

# initial exection of metmm control files
#ws.execute_controlfile("instruments/metmm/sensor_descriptions/prepare_metmm.arts")
#ws.execute_controlfile("instruments/metmm/sensor_descriptions/sensor_gmi.arts") #atms.arts")
#ws.execute_controlfile("instruments/metmm/sensor_descriptions/apply_metmm.arts")
#ws.execute_controlfile("instruments/metmm/common_metmm.arts")

# Perform some checks

In [8]:
ws.atmfields_checkedCalc( bad_partition_functions_ok = 1 )
ws.abs_xsec_agenda_checkedCalc()
ws.propmat_clearsky_agenda_checkedCalc()
ws.atmgeom_checkedCalc()
#ws.abs_lookupSetup() # not currently using lookup tables (on the fly instead)
#ws.abs_lookupCalc()

# define surface agendas -- unnecessary right now

In [9]:
#from typhon.arts.workspace import arts_agenda
#
#@arts_agenda
#def surface_fastem(ws):
#    ws.specular_losCalc()
#    ws.InterpSurfaceFieldToPosition( out = ws.surface_skin_t, field = ws.t_surface )
#    ws.surfaceFastem(wind_speed     = ws.wind_speed,
#                     wind_direction = ws.wind_direction,
#                     transmittance  = ws.transmittance)
#    
#    
#ws.TessemNNReadAscii(tessem_netv, "testdata/tessem_sav_net_V.txt")
#ws.TessemNNReadAscii(tessem_neth, "testdata/tessem_sav_net_H.txt") 
#@arts_agenda
#def surface_tessem(ws):
#    ws.specular_losCalc()
#    ws.InterpSurfaceFieldToPosition( out = ws.surface_skin_t, field = ws.t_surface )
#    ws.SurfaceTessem(wind_speed     = ws.wind_speed) # one with capital S and one without?!
#
## set surface agenda (i.e. either fastem or tessem) here:
##ws.Copy(ws.surface_rtprop_agenda, surface_fastem) # choose which emis. model to run here!
#ws.Copy(ws.surface_rtprop_agenda, surface_tessem) # choose which emis. model to run here!

# Designate surface vars

In [10]:

ws.NumericCreate("wind_speed")
ws.NumericCreate("wind_direction")
ws.wind_speed = 9.5
ws.surface_skin_t = 287.8
#ws.wind_direction = 0.0 # can adjust based on analysis data later  -- default should be 0
# -- need sensor azimuth angle, and spacecraft heading, as arts azimuth is relative to N
#ws.salinity = .035 # default is .035 anyway

# transmittance only needed for running fastem:
#ws.VectorCreate("transmittance")
#ws.transmittance  = np.ones(ws.f_grid.value.shape) # or ones, or something else?
print('wind speed, SST: ',ws.wind_speed.value,ws.surface_skin_t.value)#ws.t_surface.value.ravel()[0])

# add surface variables for jacobian calculation -- need to keep order consistent between snames/sdata
snames = ["Water skin temperature", "Wind speed", "Salinity"]
sdata = np.array([ws.surface_skin_t.value, ws.wind_speed.value, 0.035]).reshape(3,1,1)
#sdata = np.array([ws.t_surface.value.ravel()[0], ws.wind_speed.value, 0.035]).reshape(3,1,1)

# don't copy to arts variables yet... otherwise ycalc bombs due to SurfaceDummy
ws.Copy(ws.surface_props_names, snames)
ws.Copy(ws.surface_props_data, sdata)

wind speed, SST:  9.5 287.8


In [11]:
#### want to get rid of SurfaceDummy from iy_surface agenda
#AgendaSet( iy_surface_agenda__UseSurfaceRtprop ){
#  SurfaceDummy
#  iySurfaceRtpropAgenda
#}

# read tessem ascii files into arts memory
ws.TessemNNReadAscii(tessem_netv, "testdata/tessem_sav_net_V.txt")
ws.TessemNNReadAscii(tessem_neth, "testdata/tessem_sav_net_H.txt") 
@arts_agenda
def iy_surface_agendaPY(ws):
    ws.SurfaceTessem()
    ws.iySurfaceRtpropCalc()
    #ws.iySurfaceRtpropAgenda()
    # don't call SurfaceDummy()!
    
ws.Copy(ws.iy_surface_agenda, iy_surface_agendaPY) # copy python-defined agenda to ARTS

# define jacobians, run ycalc 

In [12]:

ws.ArrayOfIndexCreate("viewing_angles_1")
ws.viewing_angles_1 = [1] # index, defined in GMI metmm file
ws.ArrayOfIndexCreate("viewing_angles_2")
ws.viewing_angles_2 = [0]

ws.ArrayOfIndexCreate("channels_1") # low freq channels EIA
ws.channels_1 = csub[0]
ws.ArrayOfIndexCreate("channels_2") # high freq channels EIA
ws.channels_2 = csub[1] 

ws.Copy(ws.viewing_angles, ws.viewing_angles_1)
ws.Copy(ws.channels, ws.channels_1)


# re-execution of metmm control files ( since channels subset and angle have changed)
ws.execute_controlfile("instruments/metmm/sensor_descriptions/prepare_metmm.arts")
ws.execute_controlfile("instruments/metmm/sensor_descriptions/sensor_gmi.arts") #atms.arts")
ws.execute_controlfile("instruments/metmm/sensor_descriptions/apply_metmm.arts") # to execute CF?
#ws.VectorSetConstant(ws.transmittance, 1, 1.0) # needed for emis model, right size

ws.jacobianInit()  # initialize jacobian quantities, then add variables
ws.jacobianAddSurfaceQuantity(
    g1=ws.lat_grid, g2=ws.lon_grid, quantity=snames[0])
ws.jacobianAddSurfaceQuantity(
    g1=ws.lat_grid, g2=ws.lon_grid, quantity=snames[1])
#ws.jacobian_do = 1 # flag to activate clear-sky jacobian calculations, set to 1 when jacobianClose is called
ws.jacobianClose()

ws.cloudboxOff()
ws.cloudbox_checkedCalc()
ws.sensor_checkedCalc()

ws.yCalc()  # calculate yf and jacobian matching x
#print(ws.y.value)
#print(ws.jacobian.value)

# now run for other EIA, and append Tbs to previous EIA with yCalcAppend
ws.Copy(ws.viewing_angles, ws.viewing_angles_2)
ws.Copy(ws.channels, ws.channels_2)

ws.execute_controlfile("instruments/metmm/sensor_descriptions/prepare_metmm.arts")
ws.execute_controlfile("instruments/metmm/sensor_descriptions/sensor_gmi.arts") #atms.arts")
ws.execute_controlfile("instruments/metmm/sensor_descriptions/apply_metmm.arts") # to execute CF?
#ws.VectorSetConstant(ws.transmittance,2,1.0) # needed for emis model, right size
ws.yCalcAppend(jacobian_quantities_copy = ws.jacobian_quantities)
print(ws.y.value)
print(ws.jacobian.value)
#
#ws.Print(ws.jacobian_quantities,0)
##ws.VectorAddVector( ws.yf, ws.y, ws.y_baseline )  # add baseline term
##ws.jacobianAdjustAndTransform()
##ws.Print(ws.jacobian,0)
#ws.Print(ws.y)


[169.33739751  91.60115224 199.51783459 137.41204274 236.05000015
 220.68824268 166.40211046 271.53278258 252.60123359 283.12457811
 283.08760084 258.52720733 272.31574126]
[[ 4.01997415e-01 -3.31489309e-01]
 [-2.04717092e-01  9.72616606e-01]
 [ 3.56611617e-01 -2.85692054e-01]
 [-2.06388883e-01  8.65775351e-01]
 [ 3.18909736e-01 -1.82128148e-01]
 [ 3.06274073e-01 -2.96253346e-01]
 [-2.69686154e-01  9.08255492e-01]
 [ 2.80146348e-01 -1.44560026e-01]
 [-6.71921065e-02  3.80422530e-01]
 [ 3.63350921e-02 -3.15422169e-04]
 [ 1.71110613e-02  6.87726220e-04]
 [ 3.43197686e-12  8.45302559e-14]
 [ 6.54978335e-05  1.51464785e-06]]
